In [6]:
from ase.build import bulk, make_supercell
from ase.io.vasp import write_vasp
from ase.io.lammpsdata import write_lammps_data


def hcp_gen():
    struc = bulk("Zr", "hcp", a=3.324, c=5.168, orthorhombic=False)
    struc_orth = bulk("Zr", "hcp", a=3.324, c=5.168, orthorhombic=True)
    print(struc)

    write_vasp("POSCAR", struc, direct=True, sort=True)
    write_vasp("POSCAR_orth", struc_orth, direct=True, sort=True)
    write_lammps_data("data.pos", struc, units="metal", atom_style="atomic")  # 没有写入质量mass信息


if __name__ == "__main__":
    hcp_gen()

Atoms(symbols='Zr2', pbc=True, cell=[[3.324, 0.0, 0.0], [-1.662, 2.8786684421794737, 0.0], [0.0, 0.0, 5.168]])
